In [1]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')
sys.path.append('..')

# Maths
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ML
from soap import compute_soap_density, reshape_soaps
from soap import rrw_neighbors, make_tuples

# Utilities
import h5py
import json
from project_utils import load_structures_from_hdf5

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/helfrech/.config/matplotlib/stylelib/cosmoLarge.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  self[key] = other[key]


# TODO: load densities and plot

In [2]:
# Load SOAP cutoffs
with open('../../Processed_Data/soap_hyperparameters.json', 'r') as f:
    soap_hyperparameters = json.load(f)
    
cutoffs = soap_hyperparameters['interaction_cutoff']

In [3]:
# Load train and test indices
idxs_deem_train = np.loadtxt('../../Processed_Data/DEEM_10k/train.idxs', dtype=int)
idxs_deem_test = np.loadtxt('../../Processed_Data/DEEM_10k/test.idxs', dtype=int)

n_deem_train = idxs_deem_train.size
n_deem_test = idxs_deem_test.size
n_deem = n_deem_train + n_deem_test

In [4]:
idxs_iza_train = np.loadtxt('../../Processed_Data/IZA_226onDEEM_10k/train.idxs', dtype=int)
idxs_iza_test = np.loadtxt('../../Processed_Data/IZA_226onDEEM_10k/test.idxs', dtype=int)

In [26]:
cutoff = 6.0
coef = svc.coef_

In [27]:
df_max = np.argmax(np.matmul(soaps_test[cutoff], coef.T))
df_min = np.argmin(np.matmul(soaps_test[cutoff], coef.T))
print(df_max, df_min)

1419 111


In [28]:
np.dot(coef, soaps_test[cutoff][df_min])

array([-4.40345322])

In [29]:
np.dot(coef, soaps_test[cutoff][df_max])

array([16.25753674])

In [30]:
unscaled_soap = soaps_test[cutoff][df_max]*soaps_scale[cutoff]+soaps_center[cutoff]
w = coef * (1.0 - soaps_center[cutoff] / unscaled_soap) / soaps_scale[cutoff]
np.dot(w, unscaled_soap)

array([16.25753674])

In [31]:
soaps_scale[cutoff] = 1.0

In [32]:
# SOAP density
soap_density = compute_soap_density(12, 9, cutoff,
                                    reshape_soaps(soaps_test[cutoff][[df_max, df_min]]*soaps_scale[cutoff]+soaps_center[cutoff],
                                                  3, 12, 9),
                                    np.linspace(0, cutoff, 50), 
                                    np.linspace(-1, 1, 50),
                                    chunk_size_r=10, chunk_size_p=10)

mean_soap_density = compute_soap_density(12, 9, cutoff,
                                         reshape_soaps(np.mean(soaps_train[cutoff]*soaps_scale[cutoff]+soaps_center[cutoff], axis=0),
                                         3, 12, 9),
                                         np.linspace(0.0, cutoff, 50),
                                         np.linspace(-1, 1, 50),
                                         chunk_size_r=10, chunk_size_p=10)

density_overlap = density*soap_density

In [33]:
#real_space_w = coef * (1.0 - soaps_center[cutoff] / soaps_test[cutoff][[df_max, df_min]]) / soaps_scale[cutoff]
real_space_w = coef
real_space_density = compute_soap_density(12, 9, cutoff,
                                    reshape_soaps(real_space_w, 3, 12, 9),
                                    np.linspace(0, cutoff, 50), 
                                    np.linspace(-1, 1, 50),
                                    chunk_size_r=10, chunk_size_p=10)

density_overlap = real_space_density*soap_density

In [34]:
density_overlap.shape

(2, 3, 50, 50, 50)

In [35]:
dr = np.diff(np.linspace(0, cutoff, 50))[0]
dp = np.diff(np.linspace(-1, 1, 50))[0]
np.sum(density_overlap, axis=(1, 2, 3, 4))*dr*dr*dp

array([35.94566244, -4.53828658])

In [36]:
np.amin(soap_density), np.amax(soap_density)

(-4.039897703813097, 48.6212760239801)

In [37]:
np.amin(real_space_density), np.amax(real_space_density)

(-2.774295566411343, 2.930572774027066)

In [38]:
np.save('./real_space_soap_mean.npy', mean_soap_density)
np.save('./real_space_soap.npy', soap_density)
np.save('./real_space_weights.npy', density)

In [39]:
np.save('./nnl_soap_scaling.npy', soaps_scale)

In [40]:
deem_10k = read('../Raw_Data/DEEM_10k/DEEM_10000.xyz', index=':')
iza = read('../Raw_Data/GULP/IZA_226/IZA_OPT.xyz', index=':')
deem_10k_test = [deem_10k[i] for i in idxs_deem_test]
iza_test = [iza[i] for i in idxs_iza_test]
frames_test = iza_test + deem_10k_test
soap_neighbors, idxs_neighbors = zip(*[rrw_neighbors(frames_test[i], [14], [8, 14], cutoff, 
                                                     self_interaction=True) for i in [df_max, df_min]])

In [41]:
# 0: O-O
# 1: Si-O
# 2: Si-Si
species_pair_idx = 0
soap_idx = 0

def convert_species_idx(species_pair_idx):
    if species_pair_idx == 0:
        species_pair_label = 'OO'
    elif species_pair_idx == 1:
        species_pair_label = 'OSi'
    elif species_pair_idx == 2:
        species_pair_label = 'SiSi'
    else:
        species_pair_label = 'XX'
    return species_pair_label
 
def convert_soap_idx(soap_idx):
    if soap_idx == 0:
        soap_label = 'DEEM'
    elif soap_idx == 1:
        soap_label = 'IZA'
    else:
        soap_label = 'XXX'
    return soap_label

In [42]:
# Plot
for species_pair_idx in range(0, 3):
    species_pair_label = convert_species_idx(species_pair_idx)
    
    rx_grid, ry_grid, tz_grid = np.meshgrid(np.linspace(0, cutoff, 50), 
                                            np.linspace(0, cutoff, 50), 
                                            np.linspace(-1, 1, 50), indexing='ij')

    fig = go.Figure(data=go.Volume(x=rx_grid.flatten(),
                                   y=ry_grid.flatten(),
                                   z=tz_grid.flatten(),
                                   value=real_space_density[0][species_pair_idx].flatten(),
                                   #value=real_space_density[soap_idx][species_pair_idx].flatten(),
                                   coloraxis='coloraxis',
                                   isomin=-1.75E+0,
                                   isomax=1.75E+0,
                                   opacity=0.6,
                                   surface_count=4,
                                   caps=dict(x_show=False, y_show=False, z_show=False)))

    atom_stencil_x_deem = np.concatenate([soap_neighbors[0][center][species_pair_idx][0].flatten() 
                                     for center in range(0, len(soap_neighbors[0]))])
    atom_stencil_y_deem = np.concatenate([soap_neighbors[0][center][species_pair_idx][1].flatten()
                                     for center in range(0, len(soap_neighbors[0]))])
    atom_stencil_z_deem = np.concatenate([soap_neighbors[0][center][species_pair_idx][2].flatten()
                                     for center in range(0, len(soap_neighbors[0]))])

    atom_stencil_x_iza = np.concatenate([soap_neighbors[1][center][species_pair_idx][0].flatten() 
                                     for center in range(0, len(soap_neighbors[1]))])
    atom_stencil_y_iza = np.concatenate([soap_neighbors[1][center][species_pair_idx][1].flatten()
                                     for center in range(0, len(soap_neighbors[1]))])
    atom_stencil_z_iza = np.concatenate([soap_neighbors[1][center][species_pair_idx][2].flatten()
                                     for center in range(0, len(soap_neighbors[1]))])

    idx_x_deem = np.concatenate([idxs_neighbors[0][center][species_pair_idx][0].flatten() 
                            for center in range(0, len(soap_neighbors[0]))])
    idx_y_deem = np.concatenate([idxs_neighbors[0][center][species_pair_idx][1].flatten() 
                            for center in range(0, len(soap_neighbors[0]))])
    idx_z_deem = np.concatenate([idxs_neighbors[0][center][species_pair_idx][2].flatten() 
                            for center in range(0, len(soap_neighbors[0]))])

    idx_x_iza = np.concatenate([idxs_neighbors[1][center][species_pair_idx][0].flatten() 
                            for center in range(0, len(soap_neighbors[1]))])
    idx_y_iza = np.concatenate([idxs_neighbors[1][center][species_pair_idx][1].flatten() 
                            for center in range(0, len(soap_neighbors[1]))])
    idx_z_iza = np.concatenate([idxs_neighbors[1][center][species_pair_idx][2].flatten() 
                            for center in range(0, len(soap_neighbors[1]))])

    fig.add_trace(go.Scatter3d(x=atom_stencil_x_deem,
                               y=atom_stencil_y_deem,
                               z=atom_stencil_z_deem,
                               name=f'DEEM, {species_pair_label}',
                               mode='markers',
                               marker=dict(size=1,
                                           color='green'),
                               hovertemplate='x: %{x}<br>y: %{y}<br>z: %{z}<br>(i, j): %{text}',
                               text=['{}'.format(i) for i in zip(idx_x_deem, idx_y_deem, idx_z_deem)],
                               showlegend=True))

    fig.add_trace(go.Scatter3d(x=atom_stencil_x_iza,
                               y=atom_stencil_y_iza,
                               z=atom_stencil_z_iza,
                               name=f'IZA, {species_pair_label}',
                               mode='markers',
                               marker=dict(size=2,
                                           color='purple'),
                               hovertemplate='x: %{x}<br>y: %{y}<br>z: %{z}<br>(i, j): %{text}',
                               text=['{}'.format(i) for i in zip(idx_x_iza, idx_y_iza, idx_z_iza)],
                               showlegend=True))

    fig.update_layout(template='plotly_white',
                      scene=dict(xaxis_title='r',
                                 yaxis_title='r\'',
                                 zaxis_title='w'),
                      legend=dict(x=0.0, y=1.0,
                                  xanchor='left', yanchor='top',
                                  itemsizing='constant'),
                      coloraxis=dict(colorscale='RdBu',
                                     colorbar=dict(title='Weights')),
                      autosize=True)

    #fig.show()
    #fig.write_html(f'../Results/{cutoff}/real_space_weights-{soap_label}-{species_pair_label}.html')
    fig.write_html(f'../Results/{cutoff}/real_space_weights-IZA-DEEM-{species_pair_label}.html')
    fig.write_image(f'../Results/{cutoff}/real_space_weights-IZA-DEEM-{species_pair_label}.png')

In [43]:
# Plot
for species_pair_idx in range(0, 3):
    species_pair_label = convert_species_idx(species_pair_idx)
    for soap_idx in range(0, 2):
        soap_label = convert_soap_idx(soap_idx)
        
        rx_grid, ry_grid, tz_grid = np.meshgrid(np.linspace(0, cutoff, 50), 
                                                np.linspace(0, cutoff, 50), 
                                                np.linspace(-1, 1, 50), indexing='ij')

        fig = go.Figure(data=go.Volume(x=rx_grid.flatten(),
                                       y=ry_grid.flatten(),
                                       z=tz_grid.flatten(),
                                       value=soap_density[soap_idx][species_pair_idx].flatten(),
                                       coloraxis='coloraxis',
                                       isomin=1.0E+0,
                                       isomax=None,
                                       opacity=0.2,
                                       surface_count=20))

        atom_stencil_x = np.concatenate([soap_neighbors[soap_idx][center][species_pair_idx][0].flatten() 
                                         for center in range(0, len(soap_neighbors[soap_idx]))])
        atom_stencil_y = np.concatenate([soap_neighbors[soap_idx][center][species_pair_idx][1].flatten()
                                         for center in range(0, len(soap_neighbors[soap_idx]))])
        atom_stencil_z = np.concatenate([soap_neighbors[soap_idx][center][species_pair_idx][2].flatten()
                                         for center in range(0, len(soap_neighbors[soap_idx]))])

        idx_x = np.concatenate([idxs_neighbors[soap_idx][center][species_pair_idx][0].flatten() 
                                for center in range(0, len(soap_neighbors[soap_idx]))])
        idx_y = np.concatenate([idxs_neighbors[soap_idx][center][species_pair_idx][1].flatten() 
                                for center in range(0, len(soap_neighbors[soap_idx]))])
        idx_z = np.concatenate([idxs_neighbors[soap_idx][center][species_pair_idx][2].flatten() 
                                for center in range(0, len(soap_neighbors[soap_idx]))])


        fig.add_trace(go.Scatter3d(x=atom_stencil_x,
                                   y=atom_stencil_y,
                                   z=atom_stencil_z,
                                   name=f'{soap_label}, {species_pair_label}',
                                   mode='markers',
                                   marker=dict(size=1,
                                               color='cyan'),
                                   hovertemplate='x: %{x}<br>y: %{y}<br>z: %{z}<br>(i, j): %{text}',
                                   text=['{}'.format(i) for i in zip(idx_x, idx_y, idx_z)],
                                   showlegend=True))

        fig.update_layout(template='plotly_white',
                          scene=dict(xaxis_title='r',
                                     yaxis_title='r\'',
                                     zaxis_title='w'),
                          legend=dict(x=0.0, y=1.0,
                                      xanchor='left', yanchor='top',
                                      itemsizing='constant'),
                          coloraxis=dict(colorscale='Plasma',
                                         colorbar=dict(title='Density')),
                          autosize=True)

        #fig.show()
        fig.write_html(f'../Results/{cutoff}/real_space_soap-{soap_label}-{species_pair_label}.html')
        fig.write_image(f'../Results/{cutoff}/real_space_soap-{soap_label}-{species_pair_label}.png')

In [44]:
# Plot
for species_pair_idx in range(0, 3):
    species_pair_label = convert_species_idx(species_pair_idx)
    for soap_idx in range(0, 2):
        soap_label = convert_soap_idx(soap_idx)
        rx_grid, ry_grid, tz_grid = np.meshgrid(np.linspace(0, cutoff, 50), 
                                                np.linspace(0, cutoff, 50), 
                                                np.linspace(-1, 1, 50), indexing='ij')

        fig = go.Figure(data=go.Volume(x=rx_grid.flatten(),
                                       y=ry_grid.flatten(),
                                       z=tz_grid.flatten(),
                                       value=density_overlap[soap_idx][species_pair_idx].flatten()/10,
                                       coloraxis='coloraxis',
                                       isomin=-1.75E+0,
                                       isomax=1.75E+0,
                                       opacity=0.6,
                                       surface_count=4,
                                       caps=dict(x_show=False, y_show=False, z_show=False)))

        atom_stencil_x = np.concatenate([soap_neighbors[soap_idx][center][species_pair_idx][0].flatten() 
                                         for center in range(0, len(soap_neighbors[soap_idx]))])
        atom_stencil_y = np.concatenate([soap_neighbors[soap_idx][center][species_pair_idx][1].flatten()
                                         for center in range(0, len(soap_neighbors[soap_idx]))])
        atom_stencil_z = np.concatenate([soap_neighbors[soap_idx][center][species_pair_idx][2].flatten()
                                         for center in range(0, len(soap_neighbors[soap_idx]))])

        idx_x = np.concatenate([idxs_neighbors[soap_idx][center][species_pair_idx][0].flatten() 
                                for center in range(0, len(soap_neighbors[soap_idx]))])
        idx_y = np.concatenate([idxs_neighbors[soap_idx][center][species_pair_idx][1].flatten() 
                                for center in range(0, len(soap_neighbors[soap_idx]))])
        idx_z = np.concatenate([idxs_neighbors[soap_idx][center][species_pair_idx][2].flatten() 
                                for center in range(0, len(soap_neighbors[soap_idx]))])


        fig.add_trace(go.Scatter3d(x=atom_stencil_x,
                                   y=atom_stencil_y,
                                   z=atom_stencil_z,
                                   name=f'{soap_label}, {species_pair_label}',
                                   mode='markers',
                                   marker=dict(size=1,
                                               color='green'),
                                   hovertemplate='x: %{x}<br>y: %{y}<br>z: %{z}<br>(i, j): %{text}',
                                   text=['{}'.format(i) for i in zip(idx_x, idx_y, idx_z)],
                                   showlegend=True))

        fig.update_layout(template='plotly_white',
                          scene=dict(xaxis_title='r',
                                     yaxis_title='r\'',
                                     zaxis_title='w'),
                          legend=dict(x=0.0, y=1.0,
                                      xanchor='left', yanchor='top',
                                      itemsizing='constant'),
                          coloraxis=dict(colorscale='RdBu',
                                         colorbar=dict(title='Density*Weights')),
                          autosize=True)

        #fig.show()
        fig.write_html(f'../Results/{cutoff}/real_space_overlap-{soap_label}-{species_pair_label}.html')
        fig.write_image(f'../Results/{cutoff}/real_space_overlap-{soap_label}-{species_pair_label}.png')